# FSharp Samples

## Downloading Nuget Packages

In [1]:
#!fsharp
#r "nuget: XPlot.Plotly"
#r "nuget: FSharp.Data"
#r "nuget: MathNet.Numerics"
#r "nuget: Newtonsoft.Json" 

Installed package XPlot.Plotly version 3.0.1

Installed package MathNet.Numerics version 4.12.0

Installed package Newtonsoft.Json version 12.0.3

Installed package FSharp.Data version 3.3.3

## Reading a CSV File

In [1]:
#!fsharp
open FSharp.Data
open System.IO

let sp500 = CsvFile.Load(Path.Join(__SOURCE_DIRECTORY__, "./Data/SP500.csv" ))

// Print the prices in the HLOC format
sp500.Rows
|> Seq.take 5

index,Columns
0,"[ 2008-05-02, 1413.900024, 0.0032303750170896706 ]"
1,"[ 2008-05-05, 1407.48999, -0.0045438914840980615 ]"
2,"[ 2008-05-06, 1418.26001, 0.007622806087752565 ]"
3,"[ 2008-05-07, 1392.569946, -0.018279853115200417 ]"
4,"[ 2008-05-08, 1397.680054, 0.0036628357410659618 ]"


## Plotting a Pie Chart

In [1]:
#!fsharp
open XPlot.Plotly

[ "Pizza remaining", 3; "Pizza eaten", 5 ]
|> Chart.Pie

## Plotting a Line Plot

In [1]:
#!fsharp
open XPlot.Plotly
// Source: https://gist.github.com/theprash/7b6163d55b199359cd0dc68c477af0fa 

#r "System.Net.Http"
#r "nuget: Newtonsoft.Json"

let client = new System.Net.Http.HttpClient()
let weatherJson =
    client
        .GetStringAsync("http://www.7timer.info/bin/astro.php?lon=113.2&lat=23.1&ac=0&unit=metric&output=json&tzshift=0")
        .Result

let jObj = Newtonsoft.Json.Linq.JObject.Parse weatherJson

jObj.["dataseries"]
|> Seq.map (fun x -> x.Value<float> "timepoint", x.Value<float> "cloudcover" )
|> Seq.toArray
|> Chart.Line
|> Chart.WithXTitle "Time"
|> Chart.WithYTitle "Cloud cover"

## Plotting 2 Line Plots on One Chart

In [1]:
#!fsharp
open XPlot.Plotly
// Source: https://fslab.org/XPlot/chart/plotly-line-scatter-plots.html

let trace1 =
    Scatter(
        x = [1; 2; 3; 4],
        y = [10; 15; 13; 17]
    )

let trace2 =
    Scatter(
        x = [2; 3; 4; 5],
        y = [16; 5; 11; 9]
    )

[trace1; trace2]
|> Chart.Plot
|> Chart.WithWidth 700
|> Chart.WithHeight 500

## Plotting a Histogram

In [1]:
#!fsharp
open XPlot.Plotly
open MathNet.Numerics.Distributions

let x = 
    ContinuousUniform().Samples()
    |> Seq.take 10000
    |> Seq.toArray

Histogram(x = x) 
|> Chart.Plot
|> Chart.WithWidth 700
|> Chart.WithHeight 500

## The Metropolis Algorithm: Iterative with a Fixed Proposal Distribution

In [1]:
#!fsharp
open System
open MathNet.Numerics.Distributions
open XPlot.Plotly

let gaussianMetropolisHastingsFixedProposal (numberOfIterations: int) 
                                            (mu : double) 
                                            (sigma : double) : double list =
    let mutable states : double list = [] 
    let burnin : int = int (Math.Ceiling(float (double numberOfIterations * 0.2)))

    let proposedUniform = ContinuousUniform(mu - 5.0 * sigma, mu + 5.0 * sigma)
    let zeroOneUniform  = ContinuousUniform()
    let normal          = Normal(mu, sigma)

    // Initialize the current state.
    let mutable current = proposedUniform.Sample()

    for i in 0..numberOfIterations do

        // Proposed Move
        let proposedMove = proposedUniform.Sample()

        // Acceptance Ratio
        let currentProbability = normal.Density(current)
        let proposedProbabilty = normal.Density(proposedMove) 
        let acceptanceRatio    = Math.Min(proposedProbabilty / currentProbability, 1.0)

        // Choice
        let uniformDraw = zeroOneUniform.Sample() 

        if (uniformDraw < acceptanceRatio) then current <- proposedMove 

        // Append the current state 
        states <- states @ [current]

    states
    |> List.skip(burnin)

let mhRun = gaussianMetropolisHastingsFixedProposal 50000 0.0 1.0

Histogram(x = mhRun)
|> Chart.Plot
|> Chart.WithWidth 700
|> Chart.WithHeight 500

## Parsing and Building the Model 

### Parsing the Model Pattern Matching

In [1]:
#!fsharp
open System

type ParsedRandomVariable = 
    { Name : string; Conditionals : string list; Distribution : string; Parameters: string list; Observed : string option }

// Format: RVName [| Conditionals] ~ Distribution( Parameters ) [: observed] 
// [] -> optional
let parseFromModelPatternMatching(lineInModel : string) : ParsedRandomVariable = 
 
    // Helper fn to split the string based on a variety of type of delimiters.
    // Resultant type is a list of strings to feed in for the pattern matching.
    let splitToList (toSplit : string) (delimiters : obj) : string list = 
        let split = 
            match delimiters with
            | :? string        as s   -> toSplit.Split(s, StringSplitOptions.RemoveEmptyEntries) 
            | :? array<string> as arr -> toSplit.Split(arr, StringSplitOptions.RemoveEmptyEntries) 
            | :? array<char>   as arr -> toSplit.Split(arr, StringSplitOptions.RemoveEmptyEntries) 
            | _ -> failwithf "Splitting based on delimiters failed as it is neither a string nor an array of strings: Of Type: %A - %A" (delimiters.GetType()) toSplit
        
        Array.toList split

    match splitToList lineInModel " " with
    | nameAndConditionals :: "~" :: distributionParametersObserved ->
        // Get the name and conditionals.
        let splitNameAndConditionals = splitToList nameAndConditionals "|"
        let name = splitNameAndConditionals.[0]
        let conditionals = 
            match splitNameAndConditionals with 
            | name :: conditionals -> conditionals
            | _ -> failwithf "Pattern not found for RV Name and Conditionals - the format is: RVName|Condtionals: %A" splitNameAndConditionals

        let extractAndGetParameters (distributionNameAndParameters : string) : string * string list = 
            let splitDistributionAndParameters = splitToList distributionNameAndParameters [| "("; ")" |]
            (splitDistributionAndParameters.[0], splitToList splitDistributionAndParameters.[1] ",")
            
        match distributionParametersObserved with 

        // Case: Without Observations. Example: theta ~ Gamma(a,b)
        | distributionNameAndParameters when distributionNameAndParameters.Length = 1 ->
            let extractedDistributionAndParameters = extractAndGetParameters distributionNameAndParameters.[0]
            { Name         = name; 
              Conditionals = conditionals; 
              Distribution = fst extractedDistributionAndParameters;
              Observed     = None; 
              Parameters   = snd extractedDistributionAndParameters; }

        // Case: With Observations. Example: Y|theta ~ Poisson(theta) : observed
        | distributionNameAndParameters :: ":" :: observed ->
            let extractedDistributionAndParameters = extractAndGetParameters distributionNameAndParameters
            { Name         = name;
              Conditionals = conditionals; 
              Distribution = fst extractedDistributionAndParameters;
              Observed     = Some observed.Head; 
              Parameters   = snd extractedDistributionAndParameters; } 

        // Case: Error.
        | _ -> failwithf "Pattern not found for the model while parsing the distribution, parameters and optionally, the observed variables: %A" distributionParametersObserved 

    | _ -> failwithf "Pattern not found for the following line in the model - please check the syntax: %A" lineInModel

let printModel(model : string) : unit = 
    let parsedModel = model.Split('\n') 
                    |> Array.toList
                    |> List.map(parseFromModelPatternMatching)
                    |> List.sortBy(fun x -> x.Conditionals)
    printfn "%A" parsedModel

// Print out our simple 1-Parameter Model.
let model1 = @"theta ~ Gamma(a,b)
              Y|theta ~ Poisson(theta) : observed"
printModel(model1)

[{ Name = "theta"
   Conditionals = []
   Distribution = "Gamma"
   Parameters = ["a"; "b"]
   Observed = None }; { Name = "Y"
                        Conditionals = ["theta"]
                        Distribution = "Poisson"
                        Parameters = ["theta"]
                        Observed = Some "observed" }]

### Parameterization

In [1]:
#!fsharp
#r "nuget: Newtonsoft.Json" 

open System
open System.Collections.Generic

open Newtonsoft.Json

type ParameterList = Dictionary<string, obj>

let deserializeParameters (paramsAsString : string) : ParameterList = 
    JsonConvert.DeserializeObject<Dictionary<string, obj>>(paramsAsString)

// Parsing parameters
let parameters = "{mu0 : 0, sigma0 : 1, mu : 5, sigma : 2, lambda : 4, observed : [4,5,6,7,3,3,3,2,1]}"
printfn "%A" ( deserializeParameters parameters |> Seq.toList )

[[mu0, 0]; [sigma0, 1]; [mu, 5]; [sigma, 2]; [lambda, 4];
 [observed, [
  4,
  5,
  6,
  7,
  3,
  3,
  3,
  2,
  1
]]]

In [1]:
#!fsharp


In [1]:
#!fsharp
 